# River Mouth Water Quality Data
Collection of river mouth water quality data from Copernicus Marine Service products, starting with utilization of CMEMS WMS for data retrieval.  

_Author: Peter Kerins_  
_Created: 9 Jul 2021_  
_Environment: jupyterlab_  

Ocean Watch seeks to feature several biogeochemical parameters from the [Global Ocean Biogeochemistry Analysis and Forecast (GLOBAL_ANALYSIS_FORECAST_BIO_001_028) product](https://resources.marine.copernicus.eu/?option=com_csw&view=details&product_id=GLOBAL_ANALYSIS_FORECAST_BIO_001_028) of the [Copernicus Marine Service](https://marine.copernicus.eu/). The data will be mapped via Resource Watch. OW will also show time series of the parameters at various locations (representing outlets of rivers to the ocean).  

# Source

Presenting that time series data would typically require harvesting the information from the historical product rasters. However, the CMEMS [Pretty View](https://view-cmems.mercator-ocean.fr/GLOBAL_ANALYSIS_FORECAST_BIO_001_028) WebGIS tool displays just such time series (via the `Tools`). Furthermore, these are generated using the public-facing [WMS](https://www.ogc.org/standards/wms). The Copernicus Marine User Support Expert indicated [the relevant documentation](https://docs.geoserver.org/master/en/user/services/wms/reference.html#getfeatureinfo).

Constructing a properly formatted `GetFeatureInfo` request from scratch without examples was unsuccessful. However, spying on the Pretty View tool while making a time series request revealed [just such an example](https://nrt.cmems-du.eu/thredds/wms/global-analysis-forecast-bio-001-028-daily?SERVICE=WMS&VERSION=1.1.1&REQUEST=GetFeatureInfo&QUERY_LAYERS=chl&BBOX=53.26,14.26,53.2600001,14.2600001&HEIGHT=1&WIDTH=1&INFO_FORMAT=text/xml&SRS=EPSG:4326&X=0&Y=0&elevation=-0.49402499198913574&time=2020-07-16T12:00:00.000Z/2021-07-16T12:00:00.000Z):

```html
https://nrt.cmems-du.eu/thredds/wms/global-analysis-forecast-bio-001-028-daily?
SERVICE=WMS
&VERSION=1.1.1
&REQUEST=GetFeatureInfo
&QUERY_LAYERS=chl
&BBOX=53.26,14.26,53.2600001,14.2600001
&HEIGHT=1
&WIDTH=1
&INFO_FORMAT=text/xml
&SRS=EPSG:4326
&X=0
&Y=0
&elevation=-0.49402499198913574
&time=2020-07-16T12:00:00.000Z/2021-07-16T12:00:00.000Z
```

This query returns daily readings of `mass_concentration_of_chlorophyll_a_in_sea_water` at a depth of about half a meter for approximately the preceding year, at a location between Socotra and Yemen. 

A similar example closer to OW's needs can also be generated. [The query](https://nrt.cmems-du.eu/thredds/wms/global-analysis-forecast-bio-001-028-monthly?SERVICE=WMS&VERSION=1.1.1&REQUEST=GetFeatureInfo&QUERY_LAYERS=o2&BBOX=31.01,31.74,31.010000100000003,31.7400001&HEIGHT=1&WIDTH=1&INFO_FORMAT=text/xml&SRS=EPSG:4326&X=0&Y=0&elevation=-0.49402499198913574&time=2019-01-16T12:00:00.000Z/2021-05-16T12:00:00.000Z) for the full times series of monthly `mole_concentration_of_dissolved_molecular_oxygen_in_sea_water` values at a half-meter of depth at a pixel near the mouth of the Nile:
```html
https://nrt.cmems-du.eu/thredds/wms/global-analysis-forecast-bio-001-028-monthly?
SERVICE=WMS
&VERSION=1.1.1
&REQUEST=GetFeatureInfo
&QUERY_LAYERS=o2
&BBOX=31.01,31.74,31.010000100000003,31.7400001
&HEIGHT=1
&WIDTH=1
&INFO_FORMAT=text/xml
&SRS=EPSG:4326
&X=0
&Y=0
&elevation=-0.49402499198913574
&time=2019-01-16T12:00:00.000Z/2021-05-16T12:00:00.000Z
```

This template can be modified to accommodate OW needs.

Capabilities of service requested via:
```html
https://nrt.cmems-du.eu/thredds/wms/global-analysis-forecast-bio-001-028-monthly?
SERVICE=WMS
&VERSION=1.1.1
&REQUEST=GetCapabilities
```

Key excerpt from response:
```xml
            <GetFeatureInfo>
                <Format>image/png</Format>
                <Format>text/xml</Format>
                <DCPType>
                    <HTTP>
                        <Get>
                            <OnlineResource xlink:type="simple" xlink:href="http://nrt.cmems-du.eu/thredds/wms/global-analysis-forecast-bio-001-028-monthly" />
                        </Get>
                    </HTTP>
                </DCPType>
            </GetFeatureInfo>
```
Data can only be returned in `text/xml` format, not the preferred JSON (`application/json`).

# Parameters

## Variable
- _mole_concentration_of_dissolved_molecular_oxygen_in_sea_water (mmol m-3)_  
  - `QUERY_LAYERS=o2`  
- _mole_concentration_of_phosphate_in_sea_water (mmol m-3)_  
  - `QUERY_LAYERS=no3`  
- _mole_concentration_of_nitrate_in_sea_water (mmol m-3)_  
  - `QUERY_LAYERS=po4`  
  
Note: providing a comma-separated list of valid layers (eg `QUERY_lAYERS=o2,no3`) returns an exception (`400` response)


## Depth
- `elevation=-0.49402499198913574`
- `elevation=-1.5413750410079956`
- `elevation=-2.6456689834594727`
- `elevation=-3.8194949626922607`
- `elevation=-5.078224182128906`

## Bounding Box
`xmin`,`ymin` are just the point of selection; `xmax`, `ymax` are just minimum value plus 0.0000001

# Example Request

Oxygen
```html
https://nrt.cmems-du.eu/thredds/wms/global-analysis-forecast-bio-001-028-monthly?
SERVICE=WMS
&VERSION=1.1.1
&REQUEST=GetFeatureInfo
&QUERY_LAYERS=o2
&BBOX=31.01,31.74,31.010000100000003,31.7400001
&HEIGHT=1
&WIDTH=1
&INFO_FORMAT=info_format=text/xml
&SRS=EPSG:4326
&X=0
&Y=0
&elevation=-0.49402499198913574
&time=2019-01-16T12:00:00.000Z/2021-05-16T12:00:00.000Z
```

 # Workflow
 Utilizing the CMEMS data within an OW widget entails several steps. Prerequisite to all of these is the execution of [a Google Earth Engine script](https://code.earthengine.google.com/fd7e922bb8a4e55770dc6f3f9159df9d) to create a [river mouths dataset](https://resourcewatch.carto.com/u/wri-rw/dataset/ocn_calcs_010_target_river_mouths), which is manually uploaded to Carto:
 - Identify coordinates for each river mouth that overlap with the ocean water quality data products
     - Script: [river-mouths_adjust-coordinates.py](./river-mouths_adjust-coordinates.py)
     - This only needs to be executed once. The results are stored persistently on Carto.  
     - Valid coordinates could not be algorithmically identified for every river mouth, leaving about 500 outstanding river mouths. From these, valid coordinates were manually identified for the dozens of rivers with `ORD_FLOW` rank higher than 5, leaving over 400 rank 5 rivers without coordinates. These are to be addressed at a later date.
 - Test adjusted coordinates for validity (ie valid WMS response)
     - Script: [river-mouths_test-coordinates.py](./river-mouths_test-coordinates.py)
     - This identifies river mouths where the "valid" coordinates still do not return a proper response. These must then be manually corrected within the river mouths table.
     - Note that the WMS is not perfectly consistent, so coordinates may be successfully tested but still fail during data harvesting. This seems to occur when the adjusted coordinates are very near the edge of the land mask.
 - Add basin information to river mouth data
     - Script: [river-mouths_join-basins.py](./river-mouths_join-basins.py)
     - Joining `PFAF_ID` and `HYBAS_ID` from HydroBASINS dataset to the river mouths dataset, which is largely based on the HydroRIVERS dataset, links each river mouth to the containing basin at any desired levels (here 3 and 6).
 - For validated river mouth locations, use WMS calls to harvest and store all relevant time series data
     - Script: [river-mouths_data-collection.py](./river-mouths_data-collection.py)
     - These actual river mouth quality data are stored in [a separate table](https://resourcewatch.carto.com/u/wri-rw/dataset/ocn_020alt_chemical_concentrations).

Finally, these results must passed into the corresponding master table containing all data for all basin-type widgets. This basin table does not yet exist, and Carto does not allow direct SQL table creation, so instead the data will be organized by query, exported as a CSV file, then uploaded manually.  

```sql
INSERT INTO ow_widget_basin_demo (section, widget, identifier, hybas_id, hybas_lvl, variable_a, variable_b, date, value, unit) 
SELECT 
  'river water quality' AS section,
  'river mouth water: ' || variable AS widget,
  hyriv_id AS identifier,
  hybas_id_3 AS hybas_id,
  '3' AS hybas_lvl,
  variable AS variable_a,
  depth::text AS variable_b,
  dt AS date,
  value AS value,
  'mmol m-3' AS unit
FROM ocn_020alt_chemical_concentrations
WHERE variable='no3'
ORDER BY hyriv_id ASC, variable ASC, depth DESC, dt ASC
```

# Results

## River Mouth Locations
Original locations and effective (ie valid for WMS request) coordinates are stored in [the same table](https://resourcewatch.carto.com/u/wri-rw/dataset/ocn_calcs_010_target_river_mouths), with the initially calculated location represented by the geometry, and the validated coordinates stored in `x_valid` and `y_valid`. 

In [1]:
import os
from dotenv import dotenv_values
env_vars = dotenv_values('/home/pkerins/code/.env')
from cartoframes.auth import set_default_credentials
from cartoframes import read_carto, to_carto
import pandas as pd
CARTO_USER = env_vars['CARTO_WRI_RW_USER']
CARTO_KEY = env_vars['CARTO_WRI_RW_KEY']
set_default_credentials(username=CARTO_USER,
                        base_url="https://{user}.carto.com/".format(user=CARTO_USER),
                        api_key=CARTO_KEY)

In [2]:
read_carto('ocn_calcs_010_target_river_mouths').tail()

,cartodb_id,the_geom,main_riv,distance,upland_skm,ord_clas,ord_stra,catch_skm,length_km,dist_dn_km,...,next_down,hyriv_id,dis_av_cms,endorheic,hybas_l12,x_valid,y_valid,pfaf_id_12,hybas_id_6,hybas_id_3
5724,5675,POINT (-73.33271 -48.21388),61598266,108.362414,14019.9,1,5,1.29,1.39,0.0,...,0,61598266,184.788,0,6120023980,-74.68,-47.83,661200101000,6060023980,6030021870
5725,5721,POINT (-73.39577 -51.64509),61607961,33864.225815,8634.4,1,6,8.97,2.94,0.0,...,0,61607961,143.229,0,6120022910,-74.20,-51.54,661106011000,6060022910,6030021870
5726,3924,POINT (-126.96995 52.80455),70156165,451.358763,7767.2,1,5,8.56,4.85,0.0,...,0,70156165,136.363,0,7120018020,-127.94,51.83,783380011000,7060018020,7030014940
5727,231,POINT (-9.03723 4.99324),10856766,516.763025,2261.6,1,4,7.70,2.23,0.0,...,0,10856766,132.855,0,1120024560,-9.15,4.993243391335,145731801000,1060024320,1030023310
5728,540,POINT (9.27250 -1.87806),11048788,1932.563682,3749.2,1,5,2.79,1.77,0.0,...,0,11048788,89.213,0,1120020490,9.272504440194352,-1.9,133306010000,1060020490,1030020050


## Chemical Concentrations at River Mouths
Corrected locations are used to retrieve longitudinal data for the concentration of various chemicals at a range of depths. The results for all locations, chemicals, and depths are stored in [a single table](https://resourcewatch.carto.com/u/wri-rw/dataset/ocn_calcs_011_river_mouth_chemical_concentrations).  

In [3]:
read_carto('ocn_020alt_chemical_concentrations').tail()

,cartodb_id,the_geom,longitude,latitude,gridcentrelon,gridcentrelat,dt,variable,depth,value,hyriv_id,pfaf_id_12,hybas_id_6,hybas_id_3
2389885,2389886,None,-43.606379,59.835071,-43.5,59.75,2021-05-16 12:00:00,po4,-5.078224,0.748701,90136771,912559540000,9060006580,9030005280
2389886,2389887,None,-43.606379,59.835071,-43.5,59.75,2021-05-16 12:00:00,po4,-3.819495,0.748342,90136771,912559540000,9060006580,9030005280
2389887,2389888,None,-43.606379,59.835071,-43.5,59.75,2021-05-16 12:00:00,po4,-2.645669,0.748121,90136771,912559540000,9060006580,9030005280
2389888,2389889,None,-43.606379,59.835071,-43.5,59.75,2021-05-16 12:00:00,po4,-1.541375,0.748002,90136771,912559540000,9060006580,9030005280
2389889,2389890,None,-43.606379,59.835071,-43.5,59.75,2021-05-16 12:00:00,po4,-0.494025,0.747957,90136771,912559540000,9060006580,9030005280
